<a href="https://colab.research.google.com/github/richardlin01/CodeReview/blob/main/GiaiThich.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Khai báo các thư viện

In [ ]:
# include<FirebaseESP32.h>
# include<HardwareSerial.h>
# include<WiFi.h>

- define Macro ID và Pass Wifi
- define auth & host của Firebase

In [ ]:
// WiFi credentials
#define WIFI_SSID "TB1_ACTVN"
#define WIFI_PASSWORD "1234567@9"
// Firebase credentials
#define FIREBASE_AUTH "P1tLoliGeLjFXgjCvJWKLqTTV4UAoWKR3FIuUlFg"
#define FIREBASE_HOST "https://iotzigbee-546dd-default-rtdb.firebaseio.com/"
// Firebase database instance

Khai báo các biến trạng thái, nhiệt độ, độ ẩm, khí gas, biến thời gian hiện tại và trước đó.

In [ ]:
String path = "/SmartHomeZigbee";
FirebaseData firebaseData;
HardwareSerial zigbee(2);
String data_receiv, node, data_tx, tmp1, humi1, mq2_s, tt_den, tt_quat,s = "";
int tt ;
float temperature1;
float humidity1;
int mq2;
unsigned long curr_time, prev_time;

Chương trình chính:
- Setup baudrate truyền của Serial và của Zigbee
- Connect Wifi
- setString đẩy dữ liệu lên Firebase 2 trường LED và FAN

In [ ]:
void setup() {

  // Start serial communication
  Serial.begin(9600);
  zigbee.begin(115200, SERIAL_8N1, 16, 17);
  // Connect to WiFi network
  WiFi.begin(WIFI_SSID, WIFI_PASSWORD);
  while (WiFi.status() != WL_CONNECTED) {
    delay(10);
    Serial.println("Connecting to WiFi...");
  }
  // Initialize Firebase
  Firebase.begin(FIREBASE_HOST, FIREBASE_AUTH);
  Serial.println("Connected to firebase");
  Firebase.setString(firebaseData, path + "/LED", "off");
  Firebase.setString(firebaseData, path + "/FAN", "off");
}